In [3]:
import sys
import os

# Get the absolute path of the PROT directory
PROT_PATH = os.path.abspath(r"C:\Users\hoube\Documents\PROT")

# Add it to sys.path if it's not there
if PROT_PATH not in sys.path:
    sys.path.insert(0, PROT_PATH)

# Verify it's included
print(sys.path)

['C:\\Users\\hoube\\Documents\\PROT', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\python312.zip', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\DLLs', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\Lib', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel', '', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\Lib\\site-packages', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\Lib\\site-packages\\win32', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\hoube\\mambaforge\\envs\\sandy-devel\\Lib\\site-packages\\Pythonwin']


In [6]:
import sandy
import pandas as pd
from prot.nuclear_data import CrossSections, Covariances
from prot.sandy_interface import xs_to_df, cov_to_df
import h5py

library = "jendl_40u"
log_file = f"../data/{library}/logfile.txt"

with open(log_file, "w") as f:
    f.write("")

# Define the main HDF5 file path
hdf5_file_path = f'../data/{library}/{library}.h5'  # Single HDF5 file for all ZAIs

In [31]:
for zai in sandy.libraries.N_FILES_JENDL_40U_IAEA:
    try:
        print(f'Creating data for {zai}')
        endf = sandy.endf6.get_endf6_file(library, "xs", zai)

        temperature = 300
        err = 0.01
        ek = sandy.energy_grids.SCALE238
        errorrs = endf.get_errorr(
            err=err, 
            temperature=temperature, 
            heatr=False,
            purr=False,
            gaspr=False,
            errorr_kws=dict(ek=ek), 
            groupr_kws=dict(ek=ek)
        )

        xs = pd.DataFrame()
        cov = pd.DataFrame()

        for key in errorrs.keys():
            if key not in ['errorr34', 'errorr35']:
                xs = pd.concat([xs, xs_to_df(errorrs[key].get_xs().data)])
                cov = pd.concat([cov, cov_to_df(errorrs[key].get_cov().data)])
            elif key == 'errorr35':
                cov = pd.concat([cov, cov_to_df(errorrs[key].get_cov().data, key)])
            elif key == 'errorr34':
                log_msg = f'Covariances for angular distributions not included for {zai}'
                print(log_msg)
                with open(log_file, "a") as f:
                    f.write(log_msg + "\n")

        CrossSections(xs).to_hdf5(hdf5_file_path, zai, temperature, err)
        Covariances(cov).to_hdf5(hdf5_file_path, zai, temperature, err)

        success_msg = f'Data for {sandy.zam.zam2nuclide(zai)} written to: "{hdf5_file_path}" in group "zai_{zai}"'
        print(success_msg)
        with open(log_file, "a") as f:
            f.write(success_msg + "\n")
                
    except Exception as e:
        error_msg = f"Error processing ZAI {zai}: {str(e)}"
        print(error_msg)
        with open(log_file, "a") as f:
            f.write(error_msg + "\n")

final_msg = f'All data written to: "{hdf5_file_path}"'
print(final_msg)
with open(log_file, "a") as f:
    f.write(final_msg + "\n")

# Add error status to HDF5 file
with h5py.File(hdf5_file_path, 'a') as h5file:
        h5file.attrs['Energy grid'] = "SCALE238"
        h5file.attrs['Temperature'] = 300
        h5file.attrs['Error'] = 0.01
        h5file.attrs['processing_timestamp'] = pd.Timestamp.now().isoformat()

Creating data for 10010
Error processing ZAI 10010: Covariances not found for 10010.
Creating data for 10020
Error processing ZAI 10020: Covariances not found for 10020.
Creating data for 20030
Error processing ZAI 20030: Covariances not found for 20030.
Creating data for 20040
Error processing ZAI 20040: Covariances not found for 20040.
Creating data for 30060
Error processing ZAI 30060: Covariances not found for 30060.
Creating data for 30070
Error processing ZAI 30070: Covariances not found for 30070.
Creating data for 40090
Error processing ZAI 40090: Covariances not found for 40090.
Creating data for 50100
Data for B10 written to: "notebooks/nuclear_data/jendl_40u/jendl_40u.h5" in group "zai_50100"
Creating data for 50110
Data for B11 written to: "notebooks/nuclear_data/jendl_40u/jendl_40u.h5" in group "zai_50110"
Creating data for 60000
Error processing ZAI 60000: Covariances not found for 60000.
Creating data for 70140
Data for N14 written to: "notebooks/nuclear_data/jendl_40u/j

In [20]:
ZAI_list = [922350, 922380, 942390, 942400]

In [21]:
covariances = {str(zai): Covariances.from_hdf5(hdf5_file_path, zai) for zai in ZAI_list}

In [43]:
import h5py

with h5py.File(hdf5_file_path, 'r') as h5file:
    print(h5file.keys())
    print(h5file['30070'].keys())
    print(h5file['30070']['cross_sections'].keys())
    print(h5file['30070']['covariances'].keys())

<KeysViewHDF5 ['10010', '10020', '100200', '100210', '100220', '10030', '20030', '20040', '30060', '30070', '40090', '50100', '50110', '60000', '60130', '70140', '70150', '80160', '80170', '80180', '90190', 'zai_10010', 'zai_10020', 'zai_100200', 'zai_100210', 'zai_100220', 'zai_10030', 'zai_110220', 'zai_110230', 'zai_120240', 'zai_120250', 'zai_120260', 'zai_120270', 'zai_130260', 'zai_130270', 'zai_140280', 'zai_140290', 'zai_140300', 'zai_140310', 'zai_140320', 'zai_150310', 'zai_150320', 'zai_150330', 'zai_160320', 'zai_160330', 'zai_160340', 'zai_160350', 'zai_160360', 'zai_20030', 'zai_20040', 'zai_30060', 'zai_30070', 'zai_50100', 'zai_50110', 'zai_60000', 'zai_60130', 'zai_70140', 'zai_70150', 'zai_80160', 'zai_80170', 'zai_80180', 'zai_90190', 'zai_922350']>
<KeysViewHDF5 ['covariances', 'cross_sections']>
<KeysViewHDF5 ['axis0', 'axis1_label0', 'axis1_label1', 'axis1_label2', 'axis1_level0', 'axis1_level1', 'axis1_level2', 'block0_items', 'block0_values']>
<KeysViewHDF5 ['ax